# 빅 데이터 분석 및 시각화 개론 프로젝트
# 대중교통 승하차 데이터를 이용한 우이신설선 수요 예측
## 18조: 장민우, 표성준, 한승규
---
*** 100MB를 초과해 업로드하지 못한 BUS_STATION_BOARDING_MONTH_201708.csv 는 http://data.seoul.go.kr/openinf/fileview.jsp?infId=OA-12912 에서 다운로드 받을 수 있습니다. 파일 확장자에.exe가 붙어 다운로드 되는 경우 .exe 부분을 지워 확장자를 .csv로 강제변환하면 파일이 정상적으로 열립니다.***

---

##  1. 프로젝트 개요
- 우이신설선(2017년 9월 2일 개통) 의 예상 승하차 인원을 과거 대중교통 승하차량 통계를 이용하여 예측하는 것이 프로젝트의 주 목표입니다.
- 이미 개통하여 실제 승하차량 통계가 나와있으므로, 예측한 값과 실제 승하차량을 비교하여 오차를 분석했습니다.
### Proposal에서의 변경점
- Proposal에서 제시한 '승하차 데이터를 이용한 신규 노선의 수요 예측' 이라는 주제는 큰 틀에서 유지하였습니다. 제안 발표 당시 조교님의 제안을 받아들여 '우이신설선' 이라는 개통된지 얼마 되지 않은 노선을 타겟으로 설정하였습니다.
- 노선의 개통일은 2017년 9월 2일이므로, 2017년 8월 이전의 데이터를 사용하기로 하였습니다.


## 2. 우이신설선의 특징
- 우이신설선은 2017년 9월 2일에 개통하였습니다. 강북구 북한산우이역에서 동대문구 신설동역까지를 잇는 경전철 노선입니다.
- 일반 전철과 완전히 동일한 요금제인 수도권 통합 요금제가 적용되었으며, 이에 따라 노인 무임승차 역시 적용되어 있습니다.
- 아래의 지적도에서 확인할 수 있듯이, 환승역이 없는 위의 10개 역 (북한산우이~정릉) 은 모두 주거지역입니다. 반면 아래의 3개 환승역은 상업지구이거나, 여타 상업지구로 가기 용이한 노선들과의 환승이 이루어지고 있습니다.
![우이신설선 노선 근처 지적도](img/UIline_map.png)
- 따라서 우이신설선을 크게 두 구간 (주거구역, 환승구역)으로 나누었고, 대부분의 이용승객은 이 두 구간을 왕복하기 위해 사용하는 것으로 간주하였습니다.

## 3. 분석에 사용한 데이터
- **subwayToBus.csv** : 서울특별시 공공데이터센터에서 제공한 전철역 주변의 버스정류장을 보여주는 데이터 파일입니다(이름은 임의로 변경함). 하지만 일부 역에 대해 정확하게 표기되지 않은 버스 정류장이 있어, 조원이 직접 수정한 데이터를 사용하였습니다. 이론상으로 큰 차이는 없습니다.
- **HomeRegion.csv** : 위의 subwayToBus.csv 에는 우이신설선 근처 정류장이 표시되지 않아 따로 조사한 데이터입니다. 우이신설선 일반역 중심 반경 350m 이내의 모든 버스 정류장을 포함합니다. 다만 subwayToBus의 '근처'의 기준은 350m보다는 매우 좁기에 해당 파일에 우이신설선 역이 포함되어있어도 다시 조사할 필요는 있었을 것입니다.
- **BUS_STATION_BOARDING_MONTH_201708.csv** : 2017년 8월 한달동안 서울특별시에 면허를 둔 모든 버스 노선에서 각 정류장마다 교통카드를 이용해 승/하차한 승객의 수를 일자별로 정리한 파일입니다. (2017년 8월이 아닌 다른 월의 일부 파일은 인코딩이 utf-8이 아닌 euc-kr로 되어있습니다)
- **BuslineStops.csv** : 위의 통계 파일에는 노선의 정류장이 운행 순서가 아닌 임의로 지정되어 있었습니다. 서울특별시 대중교통센터 홈페이지에서 노선의 정류장 순차 목록과 정류소를 모두 제시하고 있어 이를 크롤링하여 csv로 정리하였습니다. 정류장명과 정류장 ID를 분리시키는 작업은 분석 중에 이루어집니다.
- **CARD_SUBWAY_MONTH_201709.csv, CARD_SUBWAY_MONTH_201711.csv** : 2017년 9월, 11월의 지하철역별 승차/하차량 통계입니다. 우이신설선 데이터가 포함되어 있습니다. (2017년 9월 1일 제외)

## 4. 분석 과정

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

### 4-1. 우이신설선 환승구역 주변의 모든 버스 정류장 검색
- 서울특별시에서 제공하는 지하철역 근처 버스 정류장 리스트(3. 참고) 를 활용하였습니다.
- 버스정류장 ID 중 0으로 시작하는 ID의 경우 데이터 정제과정 중 (숫자형 자료형으로 변환되어) 자리수가 변동이 있을 수 있으나, 버스정류장 ID는 기본적으로 다섯자리라는 사전 정보를 감안하여 처리하였습니다.
- 환승 수요가 많으므로 기존 버스 이용 시 반드시 이 역에서 환승하지 않았을 가능성이 있으므로, 실제 환승역을 기준으로 앞/뒤 한 역씩 추가적으로 조사하였습니다.

In [2]:
subwayToBus = pd.read_csv('subwayToBus.csv')
transfer_region_list = []
subway_list = ['성신여대입구','길음','한성대입구','안암','보문','창신','제기동','신설동','동묘앞']
for station in subway_list:
    transfer_region_list.append(subwayToBus.where(subwayToBus['전철역명'] == station).dropna())
transfer_region_list[1]

,전철역명,정류장명,정류장ID
16,길음,돈암동삼성아파트입구,8123.0
17,길음,길음역,8124.0
18,길음,길음역,8336.0
19,길음,길음역출구,8546.0
20,길음,길음역,8247.0
21,길음,길음역,8248.0
22,길음,길음역,8830.0
23,길음,길음전철역,8560.0
24,길음,길음전철역,8783.0
25,길음,길음뉴타운,8003.0


In [3]:
#해당 구역 안의 모든 정류장 ID를 리스트화
transfer_region = list()
for i in transfer_region_list:
    temp_list = i['정류장ID'].tolist()
    for j in temp_list:
        transfer_region.append(int(j))

### 4-2.우이신설선 주거구역 주변 반경 350m 안의 모든 버스 정류장 검색
- 관련 공식 데이터가 준비되지 않아 수동으로 조사하였습니다.
- 주거구역이므로 반경 350m(도보 5분) 이내로 기준을 정하였습니다.

In [4]:
HomeRegionData = pd.read_csv('HomeRegion.csv')
HomeRegionData

,전철역명,정류장명,정류장ID
0,북한산우이,우이동,9500
1,북한산우이,우이동차고지종점,9294
2,북한산우이,우이동차고지기점,9289
3,북한산우이,우이동도선사입구,9102
4,북한산우이,우이동도선사입구,9101
5,북한산우이,우이동도선사입구,9297
6,북한산우이,우이동도선사입구,9286
7,북한산우이,성원아파트,9833
8,북한산우이,한국사회봉사회,10712
9,북한산우이,성원아파트,9507


In [5]:
#해당 구역 안의 모든 정류장 ID를 리스트화
home_region = list()
temp_list = HomeRegionData['정류장ID'].tolist()
for j in temp_list:
    home_region.append(int(j))

### 4-3. 환승구역과 주거구역을 잇는 모든 버스 노선들을 검색
- 우이신설선 개통 직전인 2017년 8월, 모든 버스의 정류장별 승하차 인원 데이터를 통해 해당 노선이 지나가는 버스 정류소들을 확인할 수 있습니다.
- 지하철이 운행종료한 후 운행하는 N버스(심야버스)는 수요 이동의 대상이 되지 않으므로 제외하였습니다.

In [6]:
busStationBoarding8 = pd.read_csv('BUS_STATION_BOARDING_MONTH_201708.csv')
busStationBoarding = busStationBoarding8.where(busStationBoarding8['사용일자'] == 20170801).dropna()
busStationBoarding

,사용일자,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20170801.0,400,400번(염곡동~시청),102000171.0,03265,국립중앙박물관용산가족공원,130.0,108.0,20170804.0
1,20170801.0,400,400번(염곡동~시청),102000188.0,03282,한국폴리텍1대학,180.0,329.0,20170804.0
2,20170801.0,400,400번(염곡동~시청),102000179.0,03273,서빙고동주민센터,143.0,65.0,20170804.0
3,20170801.0,400,400번(염곡동~시청),102000178.0,03272,서빙고동,37.0,160.0,20170804.0
4,20170801.0,400,400번(염곡동~시청),102000180.0,03274,동빙고동,140.0,152.0,20170804.0
5,20170801.0,400,400번(염곡동~시청),102000187.0,03281,한국폴리텍1대학,179.0,210.0,20170804.0
6,20170801.0,400,400번(염곡동~시청),102000184.0,03278,보광동주민센터,338.0,109.0,20170804.0
7,20170801.0,400,400번(염곡동~시청),101000114.0,02219,남대문시장앞,15.0,108.0,20170804.0
8,20170801.0,400,400번(염곡동~시청),101000043.0,02142,롯데영프라자,56.0,103.0,20170804.0
9,20170801.0,400,400번(염곡동~시청),102000157.0,03251,신용산지하차도,21.0,107.0,20170804.0


In [7]:
def is_passing_by(region, boarding_df):
    result = boarding_df.where(boarding_df['버스정류장ARS번호'] == '0').dropna()
    for ID in region:
        if ID < 10000:
            IDstr = str(ID).zfill(5)
        else:
            IDstr = str(ID)        
        temp = boarding_df.where(boarding_df['버스정류장ARS번호'] == IDstr).dropna()
        result = result.append(temp,ignore_index=True)

    return result

transfer_region_bus = is_passing_by(transfer_region,busStationBoarding)
transfer_region_bus = transfer_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [8]:
home_region_bus = is_passing_by(home_region, busStationBoarding)
home_region_bus = home_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [9]:
common_line = []
for line in transfer_region_bus:
    if line in home_region_bus:
        if not "N" in line:
            common_line.append(line)
        
print(common_line)

['2115', '1014', '162', '1162', '성북22', '성북20', '143', '104', '151', '171', '109', '152', '7211', '110A', '1213', '153', '110B', '1114', '1164', '성북06', '성북07', '121', '1113', '1115', '120', '101', '144']


### 4-4. 해당 노선들의 순차적 정류장 목록 조사
- 해당 정보를 제공하는 웹사이트(서울특별시 대중교통센터)는 있었으나, 이를 위한 API는 존재하지 않았습니다.
- 이를 크롤링하여 하나의 csv로 정리한 후, 가공하였습니다.

In [10]:
buslineStops = pd.read_csv('BuslineStops.csv', encoding="euc_kr")
buslineStopsOriginal = buslineStops.copy()

In [11]:
busline_linenum = buslineStops.columns.tolist()
busline_linelist = list()
for linenum in busline_linenum:
    busline_linelist.append(list())
    current_line = buslineStops[linenum].tolist()
    i = 0
    for station in current_line:
        if type(station) == str:
            busline_linelist[-1].append(station.split('(')[-1].split(')')[0])
            buslineStops[linenum][i] = station.split('(')[-1].split(')')[0]
            i = i + 1

buslineStops

,152,1162,2115,1014,162,성북22,성북20,143,104,171,...,110B,7211,1213,153,110A,121,1115,1128,101,144
0,09211,08367,07418,08258,08161,08846,06591,08161,09194,08113,...,08160,12469,07195,09102,08160,09291,09211,10153,09102,09289
1,09213,08355,07338,08259,08163,08847,08465,08163,09310,08115,...,08161,35160,07194,09103,08161,09137,09213,10154,09103,09102
2,09215,08369,07435,08260,08408,08844,08479,08408,09196,08117,...,08163,12010,07233,09105,08163,09139,09215,10103,10200,09103
3,09218,08370,07433,08262,08165,08842,08494,08165,09111,08119,...,08408,12012,07231,09106,08408,09141,09218,10108,10202,09105
4,09223,08368,07296,08408,08167,08448,08507,08167,09120,08121,...,08165,12111,07229,09107,08165,09156,09223,10110,10204,09106
5,09221,08358,07295,08165,08169,08850,08522,08169,09122,08005,...,08167,12429,07227,09108,08167,09158,09307,10162,10206,09107
6,09219,08361,07294,08167,08263,08463,08539,08119,09134,08007,...,08169,12435,07225,09125,08169,09160,09274,10164,10210,09108
7,09166,08364,07430,08169,08265,08477,08554,08121,09156,08009,...,08116,12453,07223,09126,08119,09162,09295,10166,10212,09120
8,09168,08330,07292,08263,08267,08492,08571,08005,09158,01005,...,08114,12431,07221,09128,08121,09164,09276,10168,10214,09122
9,08171,08265,07290,08265,08269,08505,08584,08007,09160,01003,...,08112,12455,07218,09146,08123,09166,09303,10181,10216,09134


### 4-5. 순차 정류장 목록을 이용하여 각 버스별로 어느 구간이 우이신설선과 중복되는지 검출
- home: 주거구역 안의 정류장
- transfer: 환승구역의 정류장
- none: 어느쪽에도 해당되지 않음
- 이를 바탕으로 각 노선의 상/하행선에 대해서 home->transfer (혹은 transfer->home)으로 이동하는 모든 정류장 pair를 구한다.

In [12]:
busline_common_region = list()
busline_common_df = buslineStops.copy()
for i in range(len(busline_linenum)):
    busline_common_region.append(list())
    current_line = busline_linenum[i]
    j = 0
    for station in busline_linelist[i]:
        if int(station) in home_region:
            busline_common_region[-1].append('home')
            busline_common_df[current_line][j] = 'home'
        elif int(station) in transfer_region:
            busline_common_region[-1].append('transfer')
            busline_common_df[current_line][j] = 'transfer'
        else:
            busline_common_region[-1].append('none')
            busline_common_df[current_line][j] = 'none'
        j = j + 1

busline_common_df

,152,1162,2115,1014,162,성북22,성북20,143,104,171,...,110B,7211,1213,153,110A,121,1115,1128,101,144
0,none,none,none,none,none,none,none,none,none,none,...,none,none,none,home,none,none,none,none,home,home
1,none,none,none,none,home,home,none,home,none,none,...,none,none,none,home,none,home,none,none,home,home
2,none,none,none,none,home,home,none,home,none,home,...,home,none,none,home,home,home,none,none,home,home
3,home,none,none,none,home,home,none,home,home,home,...,home,none,none,home,home,home,home,none,none,home
4,home,none,none,home,none,none,none,none,none,none,...,home,none,none,home,home,home,home,none,none,home
5,home,none,none,home,home,none,none,home,home,none,...,none,none,none,home,none,home,home,none,none,home
6,home,none,none,none,home,none,none,home,home,transfer,...,home,none,none,home,home,home,home,none,none,home
7,home,home,none,home,home,none,none,none,home,none,...,none,none,none,none,home,home,none,none,none,none
8,none,home,none,home,none,none,none,none,home,none,...,none,none,none,none,none,home,none,none,none,home
9,none,home,none,home,transfer,transfer,none,transfer,home,none,...,none,none,none,none,transfer,home,none,none,none,home


In [13]:
onboardsection = dict()
# dictionary component is '버스번호' : {'board_u':[[ID,#],[ID,#],...], 'getoff_u':[[ID,#],[ID,#],...],
#                                     'board_d':[[ID,#],[ID,#],...], 'getoff_d':[[ID,#],[ID,#],...]}

for i in range(len(busline_linenum)):
    busline_common_region.append(list())
    current_line = busline_linenum[i]
    
    j = 0
    boardarray = []
    getoffarray = []
    linelength = len(buslineStops[current_line].dropna().index)
    for stop in busline_linelist[i]:
        if  linelength / 2 < j <= (linelength / 2) + 1:
            boardarray1, getoffarray1 = boardarray, getoffarray
            boardarray, getoffarray = [], []
        stopstatus = busline_common_df[current_line][j]
        if stopstatus == 'none':
            pass
        else:
            if len(boardarray) == 0:
                if stopstatus == 'transfer':
                    boardstatus, getoffstatus = 'transfer', 'home'
                elif stopstatus == 'home':
                    boardstatus, getoffstatus = 'home', 'transfer'
            if stopstatus == boardstatus:
                boardarray.append([buslineStops[current_line][j],j])
            elif stopstatus == getoffstatus:
                getoffarray.append([buslineStops[current_line][j],j])
        j += 1

    onboardsection[current_line] = {'board_u': boardarray1, 'getoff_u': getoffarray1,
                                    'board_d': boardarray, 'getoff_d': getoffarray}

print(onboardsection['152'])

{'board_u': [['09218', 3], ['09223', 4], ['09221', 5], ['09219', 6], ['09166', 7]], 'getoff_u': [['08003', 11], ['08007', 13], ['08179', 16], ['01243', 19], ['01044', 20]], 'board_d': [['01041', 109], ['06107', 110], ['08180', 113], ['08008', 116], ['08123', 118]], 'getoff_d': [['09167', 122], ['09220', 123], ['09222', 124], ['09224', 125], ['09217', 126]]}


### 4-6. 수요량 예측을 위한 모델링
각 노선에 대해 특정 정류장에서 탑승한 인원이 다른 정류장에서 하차할 인원 추정.
n번 정류장에서 탑승한 사람의 수를 x_n, n번 정류장에서 하차한 사람의 수를 y_n, 그리고 n번 정류장에서 탑승한 사람이 현재 탑승해있을 것이라 추정되는 사람의 수를 z_n이라 하였을 때, 하차한 사람의 수는 y_n * z_n / sum(z_n)

해당 데이터를 dic_inout에 dictionary 형태로 저장.
key 'name'에는 버스의 노선 이름을 저장한 list가 value로 저장되어있음.
각각 버스의 노선 이름을 key로 갖는 value는 특정 정류장에서 탑승한 사람이 특정 정류장에서 내렸을 때의 추정값이 이중 list로 들어가있음.

예) dic_inout['152'][0][3]
152번 노선 버스에서, 0번 정류장(첫 정류장)에서 탑승한 사람 중 3번 정류장(네 번째 정류장)에 내린 사람에 대한 추정값.

In [28]:
# 승하차 데이터 정제: 노선별로 각 정류소에서 승차한 승객과 하차한 승객의 수를 리스트로 반환하는 함수
def august_line_users(line,boarding):
    # Set boarding as True to count customers boarding.
    # Set boarding as False to count customers getting off.
    line = str(line)
    stopsN = [i for i in range(len(buslineStops[line].dropna().index))]
    stopsID = []
    for stopN in stopsN:
        stopsID.append(buslineStops[line][stopN])
    result = []
    temp = busStationBoarding8.where(busStationBoarding8['노선번호'] == line).dropna()
    if boarding:
        for stopID in stopsID:
            
            result.append(int(temp.where(temp['버스정류장ARS번호'] == stopID).dropna()['승차총승객수'].sum()))
    else:
        for stopID in stopsID:
            result.append(int(temp.where(temp['버스정류장ARS번호'] == stopID).dropna()['하차총승객수'].sum()))
    return result # 정류장별 승차(True) 혹은 하차(True) 인원수를 담은 리스트가 반환됨

print(august_line_users('1162',False))
print(august_line_users('1162',True))

[430, 10007, 10894, 2321, 900, 1175, 232, 368, 1453, 3313, 6041, 28913, 5001, 1082, 616, 1174, 1555, 2276, 5155, 1070, 1262, 3171, 3658, 3510, 4224, 5009, 11283, 3781, 2757, 10007, 10894, 2321, 900, 862]
[2864, 305, 2330, 3930, 2212, 21399, 8766, 475, 65, 7621, 2007, 1528, 6011, 159, 243, 1607, 848, 1123, 6996, 4141, 7935, 35296, 1999, 1233, 3129, 528, 526, 133, 40, 305, 2330, 3930, 2212, 1031]


In [29]:
def list_sum(_list):
    _sum = 0
    for i in _list:
        _sum = _sum + i
    return _sum
dic_inout = {'name': busline_linenum}
#list_board[탄 정류장][내린 정류장]
for i in busline_linenum:
    print("Now processing "+ str(i) + "...", end='')
    list_x = august_line_users(i, True)
    list_y = august_line_users(i, False)
    line_length = len(list_x)
    list_board = list()
    temp_list = list()
    for j in range(line_length):
        #temp_list는 해당 정류장에서 탄 사람이 현재 얼마나 타 있는지 나타내는 지표.
        list_board.append(list())
        temp_list.append(0)
    for phase in range(line_length):
        #phase 정류장을 제외한 이전까지의 탑승객 총계 산출.
        #한 정류장에 탑승한 사람이 그 정류장에서 바로 내리지는 않는다고 가정함
        now_boarding = list_sum(temp_list)
        #phase 정류장에서 해당 인원이 탑승
        temp_list[phase] = list_x[phase]
        remain = 0
        for j in range(line_length):
            if j < phase:
                # j정류장에서 탄 사람이 phase정류장에서 얼마나 내리는지 계산.
                list_board[j].append(int(list_y[phase] * temp_list[j] / now_boarding))
                temp_list[j] = temp_list[j] - list_board[j][phase]
                if temp_list[j] < 0:
                    temp_list[j]  = 0
                # 남은 인원 계산용
                remain = remain + list_board[j][phase]
            else:
                # 아직 탑승하지 않은 사람은 내릴 수도 없음
                list_board[j].append(0)
        # 나누고 남은 인원 처리.
        
        remain = list_y[phase] - remain
        former_remain = remain
        j = 0
        while remain > 0 and phase != 0:
            if j >= phase:
                j = 0
                #무한루프에 진입할 경우 강제종료
                if former_remain == remain:
                    break
                else:
                    former_remain = remain
            if temp_list[j] > 0:
                temp_list[j] = temp_list[j] - 1
                list_board[j][phase] = list_board[j][phase] + 1
                remain = remain - 1
            j = j + 1
    dic_inout[i] = list_board
    print(list_sum(temp_list) - (list_sum(list_x)-list_sum(list_y)) - list_y[0])
    print(" Done.")
    
print(dic_inout["name"])

Now processing 152...0
 Done.
Now processing 1162...18983
 Done.
Now processing 2115...609
 Done.
Now processing 1014...0
 Done.
Now processing 162...0
 Done.
Now processing 성북22...0
 Done.
Now processing 성북20...0
 Done.
Now processing 143...0
 Done.
Now processing 104...0
 Done.
Now processing 171...10222
 Done.
Now processing 109...0
 Done.
Now processing 151...0
 Done.
Now processing 1113...1093
 Done.
Now processing 1164...0
 Done.
Now processing 1114...0
 Done.
Now processing 110B...0
 Done.
Now processing 7211...0
 Done.
Now processing 1213...0
 Done.
Now processing 153...0
 Done.
Now processing 110A...0
 Done.
Now processing 121...0
 Done.
Now processing 1115...0
 Done.
Now processing 1128...0
 Done.
Now processing 101...0
 Done.
Now processing 144...0
 Done.
['152', '1162', '2115', '1014', '162', '성북22', '성북20', '143', '104', '171', '109', '151', '1113', '1164', '1114', '110B', '7211', '1213', '153', '110A', '121', '1115', '1128', '101', '144']


In [30]:
#Test for single line
line = '152'
total = 0
thisline = onboardsection[line]
for i in thisline['board_u']:
    for j in thisline['getoff_u']:
        print(str(i[1])+ " " + str(j[1]) + " " + str(dic_inout[line][i[1]][j[1]]))
        total += dic_inout[line][i[1]][j[1]]# * nowSubway_percent(j-i,section['사이역수'])
for i in thisline['board_d']:
    for j in thisline['getoff_d']:
        print(str(i[1])+ " " + str(j[1]) + " " + str(dic_inout[line][i[1]][j[1]]))
        total += dic_inout[line][i[1]][j[1]]# * nowSubway_percent(j-i,section['사이역수'])

print(total)

3 11 1822
3 13 1078
3 16 330
3 19 378
3 20 211
4 11 5207
4 13 3079
4 16 941
4 19 1079
4 20 603
5 11 3118
5 13 1844
5 16 564
5 19 647
5 20 361
6 11 1569
6 13 928
6 16 284
6 19 326
6 20 182
7 11 3235
7 13 1913
7 16 585
7 19 671
7 20 375
109 122 488
109 123 559
109 124 455
109 125 660
109 126 550
110 122 0
110 123 0
110 124 0
110 125 0
110 126 0
113 122 302
113 123 346
113 124 281
113 125 408
113 126 340
116 122 1544
116 123 1768
116 124 1439
116 125 2088
116 126 1737
118 122 764
118 123 875
118 124 713
118 125 1034
118 126 860
48541


## 5. 분석 결과
### 5-1. 4-5와 4-6 을 이용하여 우이신설선의 예상 한달 승객 계산

In [31]:
# dictionary component is '버스번호' : {'board_u':[[ID,#],[ID,#],...], 'getoff_u':[[ID,#],[ID,#],...],
#                                     'board_d':[[ID,#],[ID,#],...], 'getoff_d':[[ID,#],[ID,#],...]}
total = 0
for line in onboardsection.keys():
    thisline = onboardsection[line]
    for i in thisline['board_u']:
        for j in thisline['getoff_u']:
            total += dic_inout[line][i[1]][j[1]]# * nowSubway_percent(j-i,section['사이역수'])
    for i in thisline['board_d']:
        for j in thisline['getoff_d']:
            total += dic_inout[line][i[1]][j[1]]# * nowSubway_percent(j-i,section['사이역수'])

print(total)

378355


### 5-2. 실제 우이신설선 승하차 데이터와 검증

In [32]:
novemberSubwayUsers = pd.read_csv('CARD_SUBWAY_MONTH_201711.csv')
novemberSubwayUsers.head()
UIlineUsers11 = novemberSubwayUsers.where(novemberSubwayUsers['노선명'] == '우이신설선').dropna()
board11 = UIlineUsers11['승차총승객수'].sum()
getoff11 = UIlineUsers11['하차총승객수'].sum()
print((board11 + getoff11) / 2.0)

1172045.0


In [33]:
septemberSubwayUsers = pd.read_csv('CARD_SUBWAY_MONTH_201709.csv')
septemberSubwayUsers.head()
UIlineUsers9 = septemberSubwayUsers.where(septemberSubwayUsers['노선명'] == '우이신설선').dropna()
board9 = UIlineUsers9['승차총승객수'].sum()
getoff9 = UIlineUsers9['하차총승객수'].sum()
print((board9 + getoff9) / 2.0)

1065903.5


-9월과 11월 어느쪽과 비교해도 예상 수요치가 실제 사용량보다 훨씬 낮게 예측되었습니다.
 - 버스를 기반으로 추측한 데이터는 승차한 승객이 반드시 모두 하차하여 승차 수와 하차 수가 같다는 전제 하에 만들어진 데이터입니다.
 - 실제 우이신설선의 데이터를 보면 조금씩 차이가 발생하고 있습니다. 이를 보정하기 위하여 평균치를 통하여 비교했습니다.